In [803]:
using LowLevelFEM, LinearAlgebra, SparseArrays

In [804]:
function poissonMatrixVector(
    problem::Problem;
    coefficient::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise tensor field (node values gathered per element)
    S = nodesToElements(coefficient)
    Se = Dict(zip(S.numElem, S.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- get Lagrange shape functions for proper interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # tensor at element nodes (9×numNodes)
                    Snode = Se[elem]

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate S to Gauss point using shape functions
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Sa = reshape(Snode[9a-8:9a, 1], dim, dim)
                            Sgp .+= H[a, k] * Sa
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += ∇Na[i] * Sgp[i, j] * ∇Nb[j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


poissonMatrixVector (generic function with 1 method)

In [805]:
function gradDivMatrixF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for λ (if needed)
    λ_is_scalarfield = !(coefficient isa Number)
    if λ_is_scalarfield
        λe = nodesToElements(coefficient)
        λmap = Dict(zip(λe.numElem, λe.A))   # each entry: (numNodes×1) vector per element
    else
        λconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if λ_is_scalarfield
                        λa = λmap[elem][:, 1]  # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to GP using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate λ to GP if needed
                        λgp = λ_is_scalarfield ? dot(λa, H[:, k]) : λconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += λgp * (Fgp[i, i] * ∇Na[i]) * (Fgp[j, j] * ∇Nb[j]) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


gradDivMatrixF (generic function with 1 method)

In [806]:
function poissonMatrixSymGradF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for μ (if needed)
    μ_is_scalarfield = !(coefficient isa Number)
    if μ_is_scalarfield
        μe = nodesToElements(coefficient)
        μmap = Dict(zip(μe.numElem, μe.A))  # each entry: (numNodes×1) vector per element
    else
        μconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of F and μ)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if μ_is_scalarfield
                        μa = μmap[elem][:, 1]   # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to Gauss point using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate μ to Gauss point if needed
                        μgp = μ_is_scalarfield ? dot(μa, H[:, k]) : μconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            GradNa = zeros(dim, dim)
                            GradNb = zeros(dim, dim)
                            for i in 1:dim
                                GradNa[i, i] = ∇Na[i]
                                GradNb[i, i] = ∇Nb[i]
                            end

                            Ea = 0.5 * (Fgp' * GradNa + GradNa' * Fgp)
                            Eb = 0.5 * (Fgp' * GradNb + GradNb' * Fgp)

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += 2 * μgp * Ea[i, j] * Eb[i, j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


poissonMatrixSymGradF (generic function with 1 method)

In [807]:
function materialTangentTL_FC(
    problem::Problem;
    F::TensorField,
    C::AbstractMatrix
)
    @assert problem.dim == 3
    @assert problem.pdim == 3
    @assert size(C) == (6,6)
    @assert all(x -> (x isa Number) || (x isa ScalarField), C)

    gmsh.model.setCurrent(problem.name)

    dim  = 3
    pdim = 3
    dof  = pdim * problem.non

    # --- elementwise nodal F (9×numNodes per element)
    Fe   = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    # --- Preprocess C entries:
    # For ScalarField entries: build element->nodal-values map once.
    Centry = Matrix{Any}(undef, 6, 6)
    for I in CartesianIndices(C)
        cij = C[I]
        if cij isa ScalarField
            ce = nodesToElements(cij)
            Centry[I] = Dict(zip(ce.numElem, ce.A))  # elem => (numNodes×1) nodal values
        else
            Centry[I] = Float64(cij)
        end
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Int[]
    J = Int[]
    V = Float64[]
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # shape functions for interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Fnode = Fmap[elem]
                    Ke = zeros(pdim*numNodes, pdim*numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:3, 3k-2:3k])'
                        w    = jacDet[k] * wip[k]

                        # --- F at Gauss: Fgp = Σ_a N_a F_a
                        Fgp = zeros(3,3)
                        for a in 1:numNodes
                            Na = H[a,k]
                            Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], 3, 3)
                        end

                        # --- C at Gauss: Cgp_ij = Σ_a N_a Cij_a  (ha ScalarField)
                        Cgp = zeros(6,6)
                        for ii in 1:6, jj in 1:6
                            cij = Centry[ii,jj]
                            if cij isa Float64
                                Cgp[ii,jj] = cij
                            else
                                # dict elem => nodalValues(numNodes×1)
                                nod = cij[elem][:,1]
                                Cgp[ii,jj] = dot(nod, H[:,k])
                            end
                        end

                        # --- build B(F)  (6 × 3numNodes)
                        # Column corresponds to dof (a,j): δu_{a,j}
                        # δF = e_j ⊗ ∇N_a^T
                        # δE = sym(F^T δF) = 1/2 (F^T δF + δF^T F)
                        B = zeros(6, 3*numNodes)

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a, k]  # 3-vector

                            for j in 1:3
                                # v = F^T e_j = row j of F, as a column vector
                                v = @view Fgp[j, :]  # length-3 row view

                                # dE = 1/2 (v ⊗ ∇Na^T + ∇Na ⊗ v^T)
                                dE11 = 0.5*(v[1]*∇Na[1] + ∇Na[1]*v[1])
                                dE22 = 0.5*(v[2]*∇Na[2] + ∇Na[2]*v[2])
                                dE33 = 0.5*(v[3]*∇Na[3] + ∇Na[3]*v[3])

                                dE23 = 0.5*(v[2]*∇Na[3] + ∇Na[2]*v[3])
                                dE13 = 0.5*(v[1]*∇Na[3] + ∇Na[1]*v[3])
                                dE12 = 0.5*(v[1]*∇Na[2] + ∇Na[1]*v[2])

                                col = (a-1)*3 + j
                                B[1, col] = dE11
                                B[2, col] = dE22
                                B[3, col] = dE33
                                B[4, col] = 2*dE23
                                B[5, col] = 2*dE13
                                B[6, col] = 2*dE12
                            end
                        end

                        Ke .+= (B' * Cgp * B) * w
                    end

                    # scatter Ke
                    for a in 1:(3*numNodes), b in 1:(3*numNodes)
                        Ig = (nodeTags[div(a-1,3)+1]-1)*3 + mod(a-1,3)+1
                        Jg = (nodeTags[div(b-1,3)+1]-1)*3 + mod(b-1,3)+1
                        push!(I, Ig); push!(J, Jg); push!(V, Ke[a,b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


materialTangentTL_FC (generic function with 1 method)

In [808]:
function initialStressMatrixTL(
    problem::Problem;
    stress::TensorField   # P, S, σ – a kód nem tudja, nem is érdekli
)
    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise nodal stresses
    Se = nodesToElements(stress)
    Smap = Dict(zip(Se.numElem, Se.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for stress interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Selem = Smap[elem]
                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate stress to Gauss point (CORRECT)
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Sgp .+= Na * reshape(Selem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            # scalar geometric stiffness contribution
                            g = dot(∇Na, Sgp * ∇Nb)

                            # distribute to displacement components (i = i only)
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + i
                                Ke[ia, ib] += g * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


initialStressMatrixTL (generic function with 1 method)

In [809]:
function internalForceTL(problem::Problem, P::TensorField)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Pe = nodesToElements(P)
    Pmap = Dict(zip(Pe.numElem, Pe.A))

    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of P)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Pelem = Pmap[elem]
                    fe = zeros(pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate P to Gauss point using shape functions
                        Pgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Pgp .+= Na * reshape(Pelem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                fe[ia] += dot(Pgp[i, :], ∇Na) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:numNodes, i in 1:dim
                        Ig = (nodeTags[a] - 1) * pdim + i
                        f[Ig] += fe[(a-1)*pdim+i]
                    end
                end
            end
        end
    end

    if pdim ≠ 3
        error("dim ≠ 3 is not implemented yet")
    end
    return VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


internalForceTL (generic function with 1 method)

In [810]:
function externalTangentFollowerTL(
    problem::Problem;
    F::TensorField,
    traction_phName::AbstractString,
    t_spatial)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    Id = zeros(dim, dim)
    for i in 1:dim
        Id[i, i] = 1
    end

    I = Int[]
    J = Int[]
    V = Float64[]

    for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(traction_phName)
        elemTypes, elemTags, elemNodeTags =
            gmsh.model.mesh.getElements(edim, etag)

        for it in eachindex(elemTypes)
            et = elemTypes[it]
            _, _, order, numNodes, _, _ =
                gmsh.model.mesh.getElementProperties(et)

            ip, wip =
                gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

            # Lagrange shape functions
            _, hfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
            H = reshape(hfun, numNodes, :)

            # GradLagrange
            _, dfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
            ∇h = reshape(dfun, :, length(wip))

            for (e, elem) in enumerate(elemTags[it])
                nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                jac, jacDet, _ =
                    gmsh.model.mesh.getJacobian(elem, ip)
                Jac = reshape(jac, 3, :)

                Fnode = Fmap[elem]
                Ke = zeros(pdim * numNodes, pdim * numNodes)

                for k in eachindex(wip)
                    invJ = inv(Jac[1:dim, 3k-2:3k])'
                    w = jacDet[k] * wip[k]

                    # --- interpolate F to Gauss point using shape functions
                    Fgp = zeros(dim, dim)
                    for a in 1:numNodes
                        Na = H[a, k]
                        Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], dim, dim)
                    end

                    Jgp = det(Fgp)
                    Finv = inv(Fgp)
                    FinvT = Finv'

                    if t_spatial isa Function
                        tgp = t_spatial(0.0, 0.0, 0.0)
                    else
                        tgp = collect(Float64, t_spatial)
                    end

                    for a in 1:numNodes, b in 1:numNodes
                        ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                        # δF from δu_bj
                        for j in 1:dim
                            A = reshape(Finv[:, j], dim, 1) *
                                reshape(∇Nb, 1, dim)

                            dJFmT =
                                Jgp * FinvT * (tr(A) * Id - A')

                            dt0 = dJFmT * tgp

                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                jb = (b - 1) * pdim + j
                                Ke[ia, jb] += H[a, k] * dt0[i] * w
                            end
                        end
                    end
                end

                # scatter Ke
                for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                    Ig = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                    Jg = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                    push!(I, Ig)
                    push!(J, Jg)
                    push!(V, Ke[a, b])
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


externalTangentFollowerTL (generic function with 1 method)

In [811]:
function elementInternalForceTL(problem::Problem, et::Int, elem::Int, nodeTags::Vector{Int};
    P_of_F)

    dim = problem.dim
    pdim = problem.pdim
    @assert dim == 3 && pdim == 3

    _, _, order, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
    ip, wip = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

    # shape & grad
    _, hfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
    H = reshape(hfun, numNodes, :)
    _, dfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
    ∇h = reshape(dfun, :, length(wip))

    # coords of element nodes (reference)
    # gmsh returns node coords globally; itt feltételezem van segédfüggvényed:
    # X = getNodeCoords(problem, nodeTags) :: (3×numNodes)
    X = nodeCoordinates(problem, nodeTags)  # <-- ha nálad más a neve, írd át

    fe = zeros(pdim * numNodes)

    jac, jacDet, _ = gmsh.model.mesh.getJacobian(elem, ip)
    Jac = reshape(jac, 3, :)

    for k in eachindex(wip)
        invJ = inv(Jac[1:dim, 3k-2:3k])'
        w = jacDet[k] * wip[k]

        # GradN in reference
        GradN = zeros(3, numNodes)
        for a in 1:numNodes
            GradN[:, a] = invJ * ∇h[3a-2:3a, k]
        end

        # deformation gradient from current u (u is encoded in current nodal positions x = X + u)
        # Here we assume caller already provides current nodal positions x (or u_e).
        # We'll compute F = x ⊗ GradN + I  form:
        # F = sum_a x_a ⊗ GradN_a   (with x_a current position vector)
        # Caller will provide current x in a closure; see next function.

        # We'll leave F computation to closure by passing (X, GradN, k) if you want.
        # For simplicity: P_of_F is expected to be called in the Ke-builder where x is known.

        # This function will be used from Ke-builder where we know x.
        # So here we just return fe placeholder.
    end

    return fe
end


elementInternalForceTL (generic function with 1 method)

In [812]:
function elementMaterialTangentFD_TL(problem::Problem, et::Int, elem::Int, nodeTags::Vector{Int};
    P_of_F,
    h=1e-8)

    dim = problem.dim
    pdim = problem.pdim
    @assert dim == 3 && pdim == 3

    _, _, order, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
    ip, wip = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

    _, hfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
    H = reshape(hfun, numNodes, :)
    _, dfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
    ∇h = reshape(dfun, :, length(wip))

    X = nodeCoordinates(problem, nodeTags)  # 3×numNodes reference coords

    jac, jacDet, _ = gmsh.model.mesh.getJacobian(elem, ip)
    Jac = reshape(jac, 3, :)

    # fe(u_e) black-box
    function fe_of_ue(ue::Vector{Float64})
        # current positions x = X + u
        x = copy(X)
        for a in 1:numNodes
            x[:, a] .+= ue[(a-1)*3+1:(a-1)*3+3]
        end

        fe = zeros(3 * numNodes)

        for k in eachindex(wip)
            invJ = inv(Jac[1:dim, 3k-2:3k])'
            w = jacDet[k] * wip[k]

            # GradN in reference
            GradN = zeros(3, numNodes)
            for a in 1:numNodes
                GradN[:, a] = invJ * ∇h[3a-2:3a, k]
            end

            # F = sum_a x_a ⊗ GradN_a
            F = zeros(3, 3)
            for a in 1:numNodes
                F .+= x[:, a] * (GradN[:, a])'
            end

            P = P_of_F(F)  # 3×3

            # fe_a += (P^T * GradN_a) * w   (equiv to P : ∇N)
            # Your internalForceTL used: fe[ia] += dot(P[i,:], ∇Na) * w
            for a in 1:numNodes
                t = P * GradN[:, a]  # vector length 3
                ia = (a - 1) * 3
                fe[ia+1] += t[1] * w
                fe[ia+2] += t[2] * w
                fe[ia+3] += t[3] * w
            end
        end

        return fe
    end

    ndofe = 3 * numNodes
    Ke = zeros(ndofe, ndofe)

    ue0 = zeros(ndofe)  # caller will replace with actual element displacement when assembling
    f0 = fe_of_ue(ue0)

    for j in 1:ndofe
        ue = copy(ue0)
        ue[j] += h
        fj = fe_of_ue(ue)
        Ke[:, j] = (fj - f0) / h
    end

    return Ke, f0
end


elementMaterialTangentFD_TL (generic function with 1 method)

In [813]:
function materialTangentFD_TL(problem::Problem, u::VectorField; P_of_F, h=1e-8)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    dof = pdim * problem.non

    I = Int[]
    J = Int[]
    V = Float64[]
    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, _, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    # extract element displacement ue from global u
                    ue = zeros(3 * numNodes)
                    for a in 1:numNodes
                        g = nodeTags[a]
                        ue[(a-1)*3+1:(a-1)*3+3] .= u.a[(g-1)*3+1:(g-1)*3+3, 1]
                    end

                    # build Ke and fe at this ue
                    Ke, fe0 = elementMaterialTangentFD_TL(problem, Int(et), Int(elem), Vector{Int}(nodeTags); P_of_F=P_of_F, h=h)

                    # scatter fe
                    for a in 1:numNodes
                        g = nodeTags[a]
                        f[(g-1)*3+1:(g-1)*3+3] .+= fe0[(a-1)*3+1:(a-1)*3+3]
                    end

                    # scatter Ke
                    for a in 1:3*numNodes, b in 1:3*numNodes
                        Ig = (nodeTags[div(a - 1, 3)+1] - 1) * 3 + mod(a - 1, 3) + 1
                        Jg = (nodeTags[div(b - 1, 3)+1] - 1) * 3 + mod(b - 1, 3) + 1
                        push!(I, Ig)
                        push!(J, Jg)
                        push!(V, Ke[a, b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)

    return SystemMatrix(K, problem), VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


materialTangentFD_TL (generic function with 1 method)

In [814]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=5e3)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00122443s, CPU 0.001224s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.00030396s, CPU 0.000292s)
Info    : Meshing 3D..

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 5000.0, 0.3, 2884.6153846153843, 1923.076923076923, 4166.666666666666, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 125, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [815]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [816]:
Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

sparse([2, 3, 32, 33, 35, 36, 152, 153, 1, 31  …  157, 7, 37, 40, 46, 49, 145, 148, 154, 157], [1, 1, 1, 1, 1, 1, 1, 1, 2, 2  …  158, 159, 159, 159, 159, 159, 159, 159, 159, 159], [0.041666666666666664, -0.04166666666666666, 0.020833333333333336, -0.04166666666666666, 0.04166666666666666, -0.020833333333333332, 0.020833333333333336, -0.020833333333333332, -0.041666666666666664, -0.020833333333333336  …  -1.3877787807814457e-17, -0.020833333333333332, -0.02083333333333333, -0.08333333333333331, 0.020833333333333336, 3.469446951953614e-18, 0.020833333333333336, 0.08333333333333331, -6.938893903907228e-18, 6.938893903907228e-18], 375, 375)

In [817]:
function check_fint_Kint_consistency(
    problem,
    u::VectorField;
    bc,
    C,
    ε=1e-7,
    verbose=true
)

    # --- aktuális állapot ---
    F = u ∘ ∇
    I = unitTensor(F)
    E = (F' * F - I) / 2
    S = λ * trace(E) * I + 2μ * E
    P = F * S

    f0 = internalForceTL(problem, P).a[:, 1]

    # --- tangens ---
    #Kmat =
    #    poissonMatrixSymGradF(problem; coefficient=μ, F=F).A +
    #    gradDivMatrixF(problem; coefficient=λ, F=F).A

    #Kgeo = initialStressMatrixTL(problem; stress=P).A
    Kint = materialTangentTL_FC(problem, F=F, C=C)

    # --- perturbáció csak szabad DoF-okon ---
    fixed = constrainedDoFs(problem, bc)
    free = setdiff(1:length(f0), fixed)

    δu = zeros(length(f0))
    δu[free] .= randn(length(free))
    δu ./= norm(δu)

    # --- numerikus derivált ---
    u_pert = copy(u)
    u_pert.a[:, 1] .+= ε .* δu

    #Fp = deformationGradient(problem, u_pert)
    Fp = u_pert ∘ ∇
    Ep = (Fp' * Fp - I) / 2
    Sp = λ * trace(Ep) * I + 2μ * Ep
    Pp = Fp * Sp

    f1 = internalForceTL(problem, Pp).a[:, 1]

    df_fd = (f1 - f0) / ε
    df_lin = Kint.A * δu

    rel_err = norm(df_fd - df_lin) / max(norm(df_fd), 1e-14)

    if verbose
        println("‖df_fd‖  = ", norm(df_fd))
        println("‖Kδu‖    = ", norm(df_lin))
        println("rel err = ", rel_err)
    end

    return rel_err
end


check_fint_Kint_consistency (generic function with 1 method)

In [818]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)

load = BoundaryCondition("right", fx=10, fy=100)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, 10, 100, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [819]:
μ = mat.μ
λ = mat.λ

2884.6153846153843

In [820]:
f_ext = loadVector(prob, [load])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [821]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [822]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [823]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [824]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [825]:
P_of_F = F -> begin
    E = 0.5 * (F' * F - I(3))  # vagy a te függvényed
    S = λ * tr(E) * I(3) + 2μ * E
    F * S
end


#271 (generic function with 1 method)

In [826]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 6730.77  2884.62  2884.62     0.0      0.0      0.0
 2884.62  6730.77  2884.62     0.0      0.0      0.0
 2884.62  2884.62  6730.77     0.0      0.0      0.0
    0.0      0.0      0.0   1923.08     0.0      0.0
    0.0      0.0      0.0      0.0   1923.08     0.0
    0.0      0.0      0.0      0.0      0.0   1923.08

## Innen indul az iteráció

In [893]:
#Kμ = poissonMatrixSymGradF(prob, coefficient=μ, F=F)
#Kλ = gradDivMatrixF(prob, coefficient=λ, F=F)

#Kmat = Kμ + Kλ
Kmat = materialTangentTL_FC(prob, F=F, C=C1)

Kgeo = initialStressMatrixTL(prob, stress=S)

Kint = Kmat + Kgeo

sparse([1, 2, 3, 31, 32, 33, 34, 35, 36, 106  …  363, 364, 365, 366, 370, 371, 372, 373, 374, 375], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  375, 375, 375, 375, 375, 375, 375, 375, 375, 375], [363.48024858416676, 128.20526215159262, -112.82118028059006, 80.06091474161977, 57.69982450785314, 19.739703135131673, 78.92249371296047, -14.512040190808706, -54.070341339586726, -169.66638875054747  …  -23.560731269855356, 0.3726733611755364, -3.941277021735118, 267.2091487047664, 2.9777158909541903, 0.676188827094748, 279.6734061574727, -2.7755269477934235, -1.4121228663359346, 2357.6381807506737], 375, 375)

In [894]:
Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")

Kext = externalTangentFollowerTL(
    prob;
    F=Fright,
    traction_phName="right",
    t_spatial=[10.0, 100.0, 0.0]
)

sparse([13, 14, 15, 67, 68, 69, 70, 71, 72, 178  …  174, 175, 176, 177, 181, 182, 183, 184, 185, 186], [13, 13, 13, 13, 13, 13, 13, 13, 13, 13  …  186, 186, 186, 186, 186, 186, 186, 186, 186, 186], [-2.3257863108124287e-19, 0.3836725070273349, 0.2884636477077758, -5.704747980705066e-19, 0.18375129753657055, 0.28499101741507765, -5.3234184239061396e-18, 0.4015272035191642, 0.1422125657531526, -2.50662209437019e-18  …  1.1008615054398415e-19, -0.504445099407574, 8.282294948313409, 4.3485748077205204e-20, 0.003704139130238829, 0.001972623966367326, 4.262710519958654e-19, 0.003417691892075525, -0.0017908226323406495, 1.7100563870645138e-18], 375, 375)

In [895]:
f_int = internalForceTL(prob, P)

nodal VectorField
[-10.318250214215963; -4.2853477452488; … ; -4.6464632141862694e-10; 1.1179671077776732e-10;;]

In [896]:
K = Kint + Kext

sparse([1, 2, 3, 31, 32, 33, 34, 35, 36, 106  …  363, 364, 365, 366, 370, 371, 372, 373, 374, 375], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  375, 375, 375, 375, 375, 375, 375, 375, 375, 375], [363.48024858416676, 128.20526215159262, -112.82118028059006, 80.06091474161977, 57.69982450785314, 19.739703135131673, 78.92249371296047, -14.512040190808706, -54.070341339586726, -169.66638875054747  …  -23.560731269855356, 0.3726733611755364, -3.941277021735118, 267.2091487047664, 2.9777158909541903, 0.676188827094748, 279.6734061574727, -2.7755269477934235, -1.4121228663359346, 2357.6381807506737], 375, 375)

In [897]:
Δu = solveField(
    K,
    f_ext - f_int;
    support=[suppX]
)

fixed = constrainedDoFs(prob, [suppX])
free  = freeDoFs(prob, [suppX])
#norm(Δu.a[:, 1])
r = f_ext - f_int
println("‖r_free‖  = ", norm(r.a[free, 1]))
println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
println("‖r‖ = ", norm(r.a[:,1]))
println("‖Δu‖ = ", norm(Δu.a[:,1]))


‖r_free‖  = 8.347273241575879e-8
‖r_fixed‖ = 63.70235745385942
‖r‖ = 63.70235745385942
‖Δu‖ = 7.108384010120777e-10


In [898]:
u += Δu

nodal VectorField
[0.0; 0.0; … ; 0.0836200628093009; 0.0009252010691656534;;]

In [899]:
showDoFResults(u)
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [900]:
F = I + u ∘ ∇

elementwise TensorField
[[1.0266671942327803; 0.07549966619815879; … ; 0.00998896717817184; 0.9895143692349806;;], [1.028973072978319; 0.12254597449146265; … ; 0.010607886840190456; 0.9917886170006435;;], [1.0123499543594816; 0.14710809706815176; … ; 0.007645372758036728; 0.9948876066106151;;], [0.9991375920744343; 0.16140683687461527; … ; 0.006369197304566976; 0.9991124679407112;;], [0.9960578817771852; 0.06998858627989596; … ; 0.009719946198259798; 1.0006876029037424;;], [0.9931886972682162; 0.11919512603704954; … ; 0.009868268318930062; 0.9998059679368377;;], [0.9915941285362798; 0.14934951042740982; … ; 0.006645402522403421; 0.9990221513317704;;], [0.991791443426008; 0.15423623425160776; … ; 0.005391631979934142; 0.9985900883004509;;], [0.9632422790074203; 0.07174983764780427; … ; 0.01091961435366691; 1.0116874406519965;;], [0.9573599184357382; 0.11777497648181828; … ; 0.010350668232518928; 1.0073361086428696;;]  …  [0.9912581812112836; 0.15257237622393652; … ; -0.00664540252240342

In [901]:
E = 0.5 * (F' * F - I)
S = λ * trace(E) * I + 2μ * E
P = F * S
#P = λ * trace(F - I) * I + 2μ * (F - I)

elementwise TensorField
[[206.58748871035414; 160.38382007427617; … ; -22.05664134270901; -33.05184670144049;;], [148.77777677387536; 124.35458175610425; … ; -11.167309779115277; 4.493493593961865;;], [84.32944741022655; 98.63130700390818; … ; -6.902772343302666; 7.9595673675992415;;], [37.77175955229293; 77.23803216493774; … ; -2.3793381384865864; -5.639877609170419;;], [-9.95532229393539; 133.89391864355122; … ; -17.362388805814426; -10.198310722449852;;], [-15.293882410969958; 167.34979995978455; … ; -12.32563904627028; -13.579066369566625;;], [-11.161098505080941; 156.54312636979208; … ; -3.857264622187339; -4.024312185909514;;], [2.932869437163727; 129.7605054227814; … ; 8.845903537632154; -24.340338941670225;;], [-217.00137555724476; 121.81649307270021; … ; -18.464251486935577; -18.91236651172217;;], [-190.58656857812258; 144.05687621103343; … ; -10.19976218797297; -25.886334234286103;;]  …  [-8.45143402184687; 167.03581961861872; … ; -3.78966105073335; -6.448628670757586;;], [3.

In [902]:
err = check_fint_Kint_consistency(prob, u; C=C1, bc=[suppX])

println("Relative tangent error = ", err)


‖df_fd‖  = 2701.194165783561
‖Kδu‖    = 26.055421037953796
rel err = 1.00635421475298
Relative tangent error = 1.00635421475298


In [903]:
#gmsh.finalize()